In [33]:
import json
import re
import difflib

# ---------------------
# 파일 로드
# ---------------------
mental_disorders_path = '질병_증상_치료법_치료제.json'
medications_path = '치료제_부작용.json'
synonyms_path = 'final_synonym_dict.json'

with open(mental_disorders_path, 'r', encoding='utf-8') as f:
    mental_disorders = json.load(f)
with open(medications_path, 'r', encoding='utf-8') as f:
    medications = json.load(f)
with open(synonyms_path, 'r', encoding='utf-8') as f:
    synonyms = json.load(f)

# ---------------------
# 약물 동의어 맵 생성
# ---------------------
def build_medication_synonym_map(medications):
    med_synonyms = {}
    for med in medications:
        med_field = med.get('약물 (일반명/상품명)', '')
        match = re.match(r'(.+?)\s*\((.+?)\)', med_field)
        if match:
            names = [match.group(1).strip(), match.group(2).strip()]
        else:
            names = [med_field]
        med_synonyms[names[0]] = names
    return med_synonyms

med_synonym_map = build_medication_synonym_map(medications)
for key, values in med_synonym_map.items():
    synonyms[key] = list(set(synonyms.get(key, []) + values))

# ---------------------
# 오타 교정 함수
# ---------------------
def correct_input(user_input, known_words):
    words = user_input.split()
    corrected = []
    for word in words:
        matches = difflib.get_close_matches(word, known_words, n=1, cutoff=0.8)
        corrected.append(matches[0] if matches else word)
    return ' '.join(corrected)

# ---------------------
# 동의어 매칭
# ---------------------
def match_synonym(user_input, target_key):
    return any(re.search(rf'\b{re.escape(word)}\b', user_input) for word in synonyms.get(target_key, [target_key]))

# ---------------------
# 민감 트리거
# ---------------------
sensitive_triggers = ['죽고 싶', '자살', '극단', '해치']
med_keywords = ['약', '약물', '치료', '복용', '추천', '먹는 약']

# ---------------------
# 챗봇 메인 함수
# ---------------------
def chatbot(user_input):
    try:
        known_words = list(synonyms.keys()) + [m.get('약물 (일반명/상품명)', '') for m in medications]
        user_input_corrected = correct_input(user_input, known_words)
        user_input_clean = re.sub(r'[^\w\s]', '', user_input_corrected.strip())

        # 민감 질문 처리
        if any(trigger in user_input_clean for trigger in sensitive_triggers):
            return "⚠️ 위험한 생각이 드신다면 가까운 사람이나 전문가에게 꼭 도움을 요청하세요. 💛 당신은 혼자가 아닙니다."

        # 부작용 질문
        if '부작용' in user_input_clean:
            for med in medications:
                med_field = med.get('약물 (일반명/상품명)', '')
                med_names = med_synonym_map.get(med_field.split()[0], [med_field])
                if any(med_name in user_input_clean for med_name in med_names):
                    return f"{med_field} 부작용:\n- {med.get('부작용 상세', '부작용 정보 없음')}"
            return "앗, 해당 약물의 부작용 정보를 아직 찾지 못했어요."

        # 약물 질문 (성격장애 제외)
        if any(kw in user_input_clean for kw in med_keywords):
            for disorder in mental_disorders:
                disorder_name = disorder.get('질병', '')
                if disorder_name == "성격장애":
                    continue
                if match_synonym(user_input_clean, disorder_name):
                    treatments = disorder.get('치료제', '')
                    symptoms = disorder.get('증상', '')
                    if treatments:
                        symptoms = symptoms if symptoms else '데이터에 정보가 없습니다'
                        return f"{disorder_name}에 사용되는 약물:\n- {treatments}\n📌 특징 증상:\n- {symptoms}"

        # 증상 질문 (성격장애 제외)
        matched_diseases = []
        for disorder in mental_disorders:
            disorder_name = disorder.get('질병', '')
            symptoms = disorder.get('증상', '')
            symptom_words = re.split(r'[·, ]', symptoms)
            match_count = sum(1 for word in symptom_words if word and word in user_input_clean)
            if match_synonym(user_input_clean, disorder_name) or match_count >= 1:
                if disorder_name != '성격장애':
                    matched_diseases.append((disorder_name, symptoms))

        if matched_diseases:
            response = "🔍 관련 질병 및 특징:\n"
            for name, symptoms in matched_diseases[:3]:
                symptoms = symptoms if symptoms else '데이터에 정보가 없습니다'
                response += f"- {name}: {symptoms}\n"
            response += "더 궁금한 게 있으면 편하게 물어봐 주세요 😊"
            return response

        # fallback (성격장애 제외)
        for key, words in synonyms.items():
            if key == '성격장애':
                continue
            if any(word in user_input_clean for word in words):
                disorder = next((d for d in mental_disorders if d.get('질병') == key), {})
                symptoms = disorder.get('증상', '')
                symptoms = symptoms if symptoms else '데이터에 정보가 없습니다'
                return f"{key} 관련된 감정이 느껴져요.\n📌 특징:\n- {symptoms}\n더 궁금한 게 있으면 물어봐 주세요 😊"

        return "앗, 아직 해당 질문에 대한 답변을 준비하지 못했어요. 조금 더 구체적으로 말씀해 주세요 😊"

    except Exception as e:
        return f"앗, 오류가 발생했어요! 잠시 후 다시 시도해 주세요.\n(오류 내용: {str(e)})"


In [34]:
test_questions = [
    "플루옥세틴 부작용 뭐야?",
    "에스시탈로프람 먹으면 무슨 부작용 있어?",
    "자낙스 부작용 알려줘",
    "공황장애 약물 뭐 있어?",
    "불안할 때 먹는 약 추천해줘",
    "우울증에 쓰는 약 알려줘",
    "불면증에 좋은 약은 뭐야?",
    "ADHD 약물 추천해줄 수 있어?",
    "알츠하이머 약물 치료법 있어?",
    "양극성장애 약물 뭐가 있어?",
    "불안장애 증상 알려줘",
    "우울증 증상 뭐야?",
    "공황장애 증상 궁금해",
    "경계성 성격장애 특징은?",
    "자꾸 폭식하게 돼. 왜 그래?",
    "최근에 잠이 잘 안 와",
    "기분이 너무 오락가락해",
    "집중이 잘 안 돼",
    "요즘 무기력해",
    "나 요즘 예민해졌어",
    "요즘 너무 불안해",
    "너무 우울해",
    "최근에 너무 충동적이야",
    "나는 정신과 가야 할까?",
    "나 요즘 너무 예민해서 힘들어",
    "아무것도 하기 싫어",
    "요즘 사람 만나는 게 무서워",
    "감정 조절이 안 돼",
    "아무 생각이 없고 멍해",
    "죽고 싶다는 생각이 자꾸 들어"
]

for q in test_questions:
    print(f"Q: {q}")
    print(f"A: {chatbot(q)}\n")


Q: 플루옥세틴 부작용 뭐야?
A: 플루옥세틴 (프로작) 부작용:
- 불안, 불면, 메스꺼움, 두통, 성기능 장애, 식욕 저하, 입 마름, 발한, 설사, 떨림

Q: 에스시탈로프람 먹으면 무슨 부작용 있어?
A: 에스시탈로프람 (렉사프로) 부작용:
- 메스꺼움, 불안, 어지럼증, 졸음, 불면, 성기능 장애, 입 마름, 발한, 두통, 설사

Q: 자낙스 부작용 알려줘
A: 앗, 해당 약물의 부작용 정보를 아직 찾지 못했어요.

Q: 공황장애 약물 뭐 있어?
A: 공황장애에 사용되는 약물:
- SSRI(설트랄린, 파록세틴), 벤조디아제핀(알프라졸람)
📌 특징 증상:
- 갑작스러운 극심한 공포, 심계항진, 발한, 떨림, 숨가쁨, 흉통, 메스꺼움, 현기증, 비현실감, 죽음에 대한 공포, 발작 후 극도의 피로감

Q: 불안할 때 먹는 약 추천해줘
A: 🔍 관련 질병 및 특징:
- 강박장애: 반복되는 강박 사고(예: 오염에 대한 과도한 걱정), 강박 행동(예: 손 씻기, 확인 행동), 불안 완화를 위한 의식적 행동, 일상생활 방해
- 범불안장애: 과도하고 통제 어려운 불안·걱정, 안절부절, 피로, 집중 어려움, 근육 긴장, 수면 문제, 일상생활·사회·직업적 기능에 지장
- 불안장애: 과도한 불안·걱정, 안절부절, 긴장, 피로, 집중력 저하, 근육 긴장, 수면 문제, 일상생활·사회·직업 기능 저하
더 궁금한 게 있으면 편하게 물어봐 주세요 😊

Q: 우울증에 쓰는 약 알려줘
A: 우울증에 사용되는 약물:
- SSRI, SNRI, 미르타자핀, 부프로피온
📌 특징 증상:
- 지속적 우울감, 흥미·즐거움 상실, 체중·식욕 변화, 수면 문제, 피로감, 무가치감, 집중력 저하, 자살 생각

Q: 불면증에 좋은 약은 뭐야?
A: 🔍 관련 질병 및 특징:
- 노인우울증: 노년층에서 나타나는 우울감, 의욕·관심 상실, 불면 또는 과다수면, 피로감, 집중력·기억력 저하, 식욕·체중 변화, 자살 생각, 신체 증상 호소
- 수면장애: 불면증, 과다수면, 수면무호흡증, 주기적 사지 운동, 기면

In [75]:
import json
import re
import difflib

# ---------------------
# 파일 로드
# ---------------------
with open('질병_증상_치료법_치료제.json', 'r', encoding='utf-8') as f:
    mental_disorders = json.load(f)
with open('치료제_부작용.json', 'r', encoding='utf-8') as f:
    medications = json.load(f)
with open('final_synonym_dict.json', 'r', encoding='utf-8') as f:
    synonyms = json.load(f)

# ---------------------
# 성격장애 과잉 동의어 정리
# ---------------------
if '성격장애' in synonyms:
    synonyms['성격장애'] = [w for w in synonyms['성격장애'] if w not in ['불안', '충동']]

# ---------------------
# 약물 동의어 맵 생성
# ---------------------
def build_medication_synonym_map(medications):
    med_synonyms = {}
    for med in medications:
        med_field = med.get('약물 (일반명/상품명)', '')
        match = re.match(r'(.+?)\s*\((.+?)\)', med_field)
        if match:
            names = [match.group(1).strip(), match.group(2).strip()]
        else:
            names = [med_field]
        med_synonyms[names[0]] = names
    return med_synonyms

med_synonym_map = build_medication_synonym_map(medications)
for key, values in med_synonym_map.items():
    synonyms[key] = list(set(synonyms.get(key, []) + values))

# ---------------------
# 오타 교정 함수
# ---------------------
def correct_input(user_input, known_words):
    words = user_input.split()
    corrected = []
    for word in words:
        matches = difflib.get_close_matches(word, known_words, n=1, cutoff=0.85)
        corrected.append(matches[0] if matches else word)
    return ' '.join(corrected)

# ---------------------
# 동의어 매칭
# ---------------------
def match_synonym(user_input, target_key):
    return any(re.search(rf'\b{re.escape(word)}\b', user_input) for word in synonyms.get(target_key, [target_key]))

# ---------------------
# 민감 트리거
# ---------------------
sensitive_triggers = ['죽고 싶', '자살', '극단', '해치']

# ---------------------
# 챗봇 메인 함수
# ---------------------
def chatbot(user_input):
    try:
        known_words = list(synonyms.keys()) + [m.get('약물 (일반명/상품명)', '') for m in medications]
        user_input_corrected = correct_input(user_input, known_words)
        user_input_clean = re.sub(r'[^\w\s]', '', user_input_corrected.strip())

        # 민감 질문 처리
        if any(trigger in user_input_clean for trigger in sensitive_triggers):
            return "⚠️ 위험한 생각이 드신다면 가까운 사람이나 전문가에게 꼭 도움을 요청하세요. 💛 당신은 혼자가 아닙니다."

        # 부작용 질문
        if '부작용' in user_input_clean:
            for med in medications:
                med_field = med.get('약물 (일반명/상품명)', '')
                med_names = med_synonym_map.get(med_field.split()[0], [med_field])
                if any(med_name in user_input_clean for med_name in med_names):
                    return f"{med_field} 부작용:\n- {med.get('부작용 상세', '부작용 정보 없음')}"
            return "앗, 해당 약물의 부작용 정보를 아직 찾지 못했어요."

        # 약물 질문 (성격장애 제외)
        for disorder in mental_disorders:
            disorder_name = disorder.get('질병', '')
            if disorder_name == "성격장애":
                continue
            if match_synonym(user_input_clean, disorder_name) and disorder.get('치료제'):
                meds = ', '.join(disorder['치료제']) if isinstance(disorder['치료제'], list) else disorder['치료제']
                symptoms = disorder.get('증상', '정보 없음')
                return f"💊 질병별 약물 정보:\n- {disorder_name}\n  약물: {meds}\n  증상: {symptoms}\n더 궁금한 게 있으면 물어봐 주세요 😊"

        # 증상 질문
        matched_diseases = []
        for disorder in mental_disorders:
            disorder_name = disorder.get('질병', '')
            if disorder_name == "성격장애":
                continue
            symptoms = disorder.get('증상', '')
            symptom_words = re.split(r'[·, ]', symptoms)
            match_count = sum(1 for word in symptom_words if word and word in user_input_clean)
            if match_synonym(user_input_clean, disorder_name) or match_count >= 1:
                matched_diseases.append((disorder_name, symptoms))

        if matched_diseases:
            response = "🔍 관련 질병 및 특징:\n"
            for name, symptoms in matched_diseases[:3]:
                response += f"- {name}: {symptoms}\n"
            response += "더 궁금한 게 있으면 편하게 물어봐 주세요 😊"
            return response

        # fallback (성격장애 제외)
        for key, words in synonyms.items():
            if key == '성격장애':
                continue
            if any(word in user_input_clean for word in words):
                disorder = next((d for d in mental_disorders if d.get('질병') == key), {})
                symptoms = disorder.get('증상', '데이터에 정보가 없습니다')
                return f"{key} 관련된 감정이 느껴져요.\n📌 특징:\n- {symptoms}\n더 궁금한 게 있으면 물어봐 주세요 😊"

        return "앗, 아직 해당 질문에 대한 답변을 준비하지 못했어요. 조금 더 구체적으로 말씀해 주세요 😊"

    except Exception as e:
        return f"앗, 오류가 발생했어요! 잠시 후 다시 시도해 주세요.\n(오류 내용: {str(e)})"


In [76]:
test_questions = [
    "플루옥세틴 부작용 뭐야?",
    "에스시탈로프람 먹으면 무슨 부작용 있어?",
    "자낙스 부작용 알려줘",
    "공황장애 약물 뭐 있어?",
    "불안할 때 먹는 약 추천해줘",
    "우울증에 쓰는 약 알려줘",
    "불면증에 좋은 약은 뭐야?",
    "ADHD 약물 추천해줄 수 있어?",
    "알츠하이머 약물 치료법 있어?",
    "양극성장애 약물 뭐가 있어?",
    "불안장애 증상 알려줘",
    "우울증 증상 뭐야?",
    "공황장애 증상 궁금해",
    "경계성 성격장애 특징은?",
    "자꾸 폭식하게 돼. 왜 그래?",
    "최근에 잠이 잘 안 와",
    "기분이 너무 오락가락해",
    "집중이 잘 안 돼",
    "요즘 무기력해",
    "나 요즘 예민해졌어",
    "요즘 너무 불안해",
    "너무 우울해",
    "최근에 너무 충동적이야",
    "나는 정신과 가야 할까?",
    "나 요즘 너무 예민해서 힘들어",
    "아무것도 하기 싫어",
    "요즘 사람 만나는 게 무서워",
    "감정 조절이 안 돼",
    "아무 생각이 없고 멍해",
    "죽고 싶다는 생각이 자꾸 들어"
]

for q in test_questions:
    print(f"Q: {q}")
    print(f"A: {chatbot(q)}\n")


Q: 플루옥세틴 부작용 뭐야?
A: 플루옥세틴 (프로작) 부작용:
- 불안, 불면, 메스꺼움, 두통, 성기능 장애, 식욕 저하, 입 마름, 발한, 설사, 떨림

Q: 에스시탈로프람 먹으면 무슨 부작용 있어?
A: 에스시탈로프람 (렉사프로) 부작용:
- 메스꺼움, 불안, 어지럼증, 졸음, 불면, 성기능 장애, 입 마름, 발한, 두통, 설사

Q: 자낙스 부작용 알려줘
A: 앗, 해당 약물의 부작용 정보를 아직 찾지 못했어요.

Q: 공황장애 약물 뭐 있어?
A: 💊 질병별 약물 정보:
- 공황장애
  약물: SSRI(설트랄린, 파록세틴), 벤조디아제핀(알프라졸람)
  증상: 갑작스러운 극심한 공포, 심계항진, 발한, 떨림, 숨가쁨, 흉통, 메스꺼움, 현기증, 비현실감, 죽음에 대한 공포, 발작 후 극도의 피로감
더 궁금한 게 있으면 물어봐 주세요 😊

Q: 불안할 때 먹는 약 추천해줘
A: 🔍 관련 질병 및 특징:
- 강박장애: 반복되는 강박 사고(예: 오염에 대한 과도한 걱정), 강박 행동(예: 손 씻기, 확인 행동), 불안 완화를 위한 의식적 행동, 일상생활 방해
- 범불안장애: 과도하고 통제 어려운 불안·걱정, 안절부절, 피로, 집중 어려움, 근육 긴장, 수면 문제, 일상생활·사회·직업적 기능에 지장
- 불안장애: 과도한 불안·걱정, 안절부절, 긴장, 피로, 집중력 저하, 근육 긴장, 수면 문제, 일상생활·사회·직업 기능 저하
더 궁금한 게 있으면 편하게 물어봐 주세요 😊

Q: 우울증에 쓰는 약 알려줘
A: 💊 질병별 약물 정보:
- 우울증
  약물: SSRI, SNRI, 미르타자핀, 부프로피온
  증상: 지속적 우울감, 흥미·즐거움 상실, 체중·식욕 변화, 수면 문제, 피로감, 무가치감, 집중력 저하, 자살 생각
더 궁금한 게 있으면 물어봐 주세요 😊

Q: 불면증에 좋은 약은 뭐야?
A: 🔍 관련 질병 및 특징:
- 노인우울증: 노년층에서 나타나는 우울감, 의욕·관심 상실, 불면 또는 과다수면, 피로감, 집중력·기억력 저하, 식욕·체중 변화, 자살 생

In [109]:
import json
import re
import difflib
import logging

# ---------------------
# 로깅 설정
# ---------------------
logging.basicConfig(level=logging.INFO, format='[%(levelname)s] %(message)s')

# ---------------------
# 파일 로드
# ---------------------
mental_disorders_path = '질병_증상_치료법_치료제.json'
medications_path = '치료제_부작용.json'
synonyms_path = 'final_synonym_dict.json'

with open(mental_disorders_path, 'r', encoding='utf-8') as f:
    mental_disorders = json.load(f)
with open(medications_path, 'r', encoding='utf-8') as f:
    medications = json.load(f)
with open(synonyms_path, 'r', encoding='utf-8') as f:
    synonyms = json.load(f)

# ---------------------
# 약물 동의어 맵 생성
# ---------------------
def build_medication_synonym_map(medications):
    med_synonyms = {}
    for med in medications:
        med_field = med.get('약물 (일반명/상품명)', '')
        match = re.match(r'(.+?)\s*\((.+?)\)', med_field)
        if match:
            names = [match.group(1).strip(), match.group(2).strip()]
        else:
            names = [med_field.strip()]
        med_synonyms[names[0]] = names
    return med_synonyms

med_synonym_map = build_medication_synonym_map(medications)
for key, values in med_synonym_map.items():
    synonyms[key] = list(set(synonyms.get(key, []) + values))

# ---------------------
# 오타 교정 함수
# ---------------------
def correct_input(user_input, known_words):
    words = user_input.split()
    corrected = []
    for word in words:
        matches = difflib.get_close_matches(word, known_words, n=1, cutoff=0.8)
        corrected_word = matches[0] if matches else word
        corrected.append(corrected_word)
    corrected_input = ' '.join(corrected)
    logging.info(f'Original input: {user_input}')
    logging.info(f'Corrected input: {corrected_input}')
    return corrected_input

# ---------------------
# 동의어 매칭
# ---------------------
def match_synonym(user_input, target_key):
    return any(re.search(rf'\b{re.escape(word)}\b', user_input) for word in synonyms.get(target_key, [target_key]))

# ---------------------
# 민감 트리거
# ---------------------
sensitive_triggers = ['죽고 싶', '자살', '극단', '해치']
med_keywords = ['약', '약물', '치료', '복용', '추천', '먹는 약']

# ---------------------
# 챗봇 메인 함수
# ---------------------
def chatbot(user_input):
    try:
        known_words = list(synonyms.keys()) + [m.get('약물 (일반명/상품명)', '') for m in medications]
        user_input_corrected = correct_input(user_input, known_words)
        user_input_clean = re.sub(r'[^\w\s]', '', user_input_corrected.strip())
        logging.info(f'Clean input: {user_input_clean}')

        # 민감 질문 처리
        if any(trigger in user_input_clean for trigger in sensitive_triggers):
            return "⚠️ 위험한 생각이 드신다면 가까운 사람이나 전문가에게 꼭 도움을 요청하세요. 💛 당신은 혼자가 아닙니다."

        # 부작용 질문
        if '부작용' in user_input_clean:
            for med in medications:
                med_field = med.get('약물 (일반명/상품명)', '')
                med_names = med_synonym_map.get(med_field.split()[0], [med_field])
                if any(med_name in user_input_clean for med_name in med_names):
                    side_effects = med.get('부작용 상세', '부작용 정보 없음')
                    return f"💊 {med_field} 부작용:\n- {side_effects}"
            return "앗, 해당 약물의 부작용 정보를 아직 찾지 못했어요."

        # 약물 질문
        if any(kw in user_input_clean for kw in med_keywords):
            for disorder in mental_disorders:
                disorder_name = disorder.get('질병', '')
                if disorder_name == '성격장애':  # 성격장애 우선 제외
                    continue
                if match_synonym(user_input_clean, disorder_name):
                    treatments = disorder.get('치료제', '')
                    symptoms = disorder.get('증상', '')
                    treatments_msg = treatments if treatments else '데이터에 정보가 없습니다'
                    symptoms_msg = symptoms if symptoms else '데이터에 정보가 없습니다'
                    return f"💊 {disorder_name}에 사용되는 약물:\n- {treatments_msg}\n📌 특징 증상:\n- {symptoms_msg}"

        # 증상 질문
        matched_diseases = []
        for disorder in mental_disorders:
            disorder_name = disorder.get('질병', '')
            symptoms = disorder.get('증상', '')
            symptom_words = re.split(r'[·, ]', symptoms)
            match_count = sum(1 for word in symptom_words if word and word in user_input_clean)
            if match_synonym(user_input_clean, disorder_name) or match_count >= 1:
                if disorder_name != '성격장애':
                    matched_diseases.append((disorder_name, symptoms))

        if matched_diseases:
            response = "🔍 관련 질병 및 특징:\n"
            for name, symptoms in matched_diseases[:3]:
                symptoms_msg = symptoms if symptoms else '데이터에 정보가 없습니다'
                response += f"- {name}: {symptoms_msg}\n"
            response += "더 궁금한 게 있으면 편하게 물어봐 주세요 😊"
            return response

        # fallback 최종 레벨만 성격 fallback
        personality = next((d for d in mental_disorders if d.get('질병') == '성격장애'), None)
        if personality:
            symptoms = personality.get('증상', '')
            symptoms_msg = symptoms if symptoms else '데이터에 정보가 없습니다'
            return f"💬 현재 질문에 맞는 정확한 데이터를 찾지 못했어요.\n성격장애 정보 참고:\n- {symptoms_msg}\n더 궁금한 게 있으면 물어봐 주세요 😊"

    except Exception as e:
        logging.error(f'Error: {e}')
        return f"앗, 오류가 발생했어요! 잠시 후 다시 시도해 주세요.\n(오류 내용: {str(e)})"


In [110]:
test_questions = [
    "플루옥세틴 부작용 뭐야?",
    "에스시탈로프람 먹으면 무슨 부작용 있어?",
    "자낙스 부작용 알려줘",
    "공황장애 약물 뭐 있어?",
    "불안할 때 먹는 약 추천해줘",
    "우울증에 쓰는 약 알려줘",
    "불면증에 좋은 약은 뭐야?",
    "ADHD 약물 추천해줄 수 있어?",
    "알츠하이머 약물 치료법 있어?",
    "양극성장애 약물 뭐가 있어?",
    "불안장애 증상 알려줘",
    "우울증 증상 뭐야?",
    "공황장애 증상 궁금해",
    "경계성 성격장애 특징은?",
    "자꾸 폭식하게 돼. 왜 그래?",
    "최근에 잠이 잘 안 와",
    "기분이 너무 오락가락해",
    "집중이 잘 안 돼",
    "요즘 무기력해",
    "나 요즘 예민해졌어",
    "요즘 너무 불안해",
    "너무 우울해",
    "최근에 너무 충동적이야",
    "나는 정신과 가야 할까?",
    "나 요즘 너무 예민해서 힘들어",
    "아무것도 하기 싫어",
    "요즘 사람 만나는 게 무서워",
    "감정 조절이 안 돼",
    "아무 생각이 없고 멍해",
    "죽고 싶다는 생각이 자꾸 들어"
]

for q in test_questions:
    print(f"Q: {q}")
    print(f"A: {chatbot(q)}\n")


Q: 플루옥세틴 부작용 뭐야?
A: 💊 플루옥세틴 (프로작) 부작용:
- 불안, 불면, 메스꺼움, 두통, 성기능 장애, 식욕 저하, 입 마름, 발한, 설사, 떨림

Q: 에스시탈로프람 먹으면 무슨 부작용 있어?
A: 💊 에스시탈로프람 (렉사프로) 부작용:
- 메스꺼움, 불안, 어지럼증, 졸음, 불면, 성기능 장애, 입 마름, 발한, 두통, 설사

Q: 자낙스 부작용 알려줘
A: 앗, 해당 약물의 부작용 정보를 아직 찾지 못했어요.

Q: 공황장애 약물 뭐 있어?
A: 💊 공황장애에 사용되는 약물:
- SSRI(설트랄린, 파록세틴), 벤조디아제핀(알프라졸람)
📌 특징 증상:
- 갑작스러운 극심한 공포, 심계항진, 발한, 떨림, 숨가쁨, 흉통, 메스꺼움, 현기증, 비현실감, 죽음에 대한 공포, 발작 후 극도의 피로감

Q: 불안할 때 먹는 약 추천해줘
A: 🔍 관련 질병 및 특징:
- 강박장애: 반복되는 강박 사고(예: 오염에 대한 과도한 걱정), 강박 행동(예: 손 씻기, 확인 행동), 불안 완화를 위한 의식적 행동, 일상생활 방해
- 범불안장애: 과도하고 통제 어려운 불안·걱정, 안절부절, 피로, 집중 어려움, 근육 긴장, 수면 문제, 일상생활·사회·직업적 기능에 지장
- 불안장애: 과도한 불안·걱정, 안절부절, 긴장, 피로, 집중력 저하, 근육 긴장, 수면 문제, 일상생활·사회·직업 기능 저하
더 궁금한 게 있으면 편하게 물어봐 주세요 😊

Q: 우울증에 쓰는 약 알려줘
A: 💊 우울증에 사용되는 약물:
- SSRI, SNRI, 미르타자핀, 부프로피온
📌 특징 증상:
- 지속적 우울감, 흥미·즐거움 상실, 체중·식욕 변화, 수면 문제, 피로감, 무가치감, 집중력 저하, 자살 생각

Q: 불면증에 좋은 약은 뭐야?
A: 🔍 관련 질병 및 특징:
- 노인우울증: 노년층에서 나타나는 우울감, 의욕·관심 상실, 불면 또는 과다수면, 피로감, 집중력·기억력 저하, 식욕·체중 변화, 자살 생각, 신체 증상 호소
- 수면장애: 불면증, 과다수면, 수면무호흡증, 주기적 사

In [159]:
import json
import re
import difflib
import random
import logging

# -----------------------------
# 로깅 설정
# -----------------------------
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# -----------------------------
# 파일 경로
# -----------------------------
mental_disorders_path = '질병_증상_치료법_치료제.json'
medications_path = '치료제_부작용.json'
synonyms_path = 'final_synonym_dict.json'

# -----------------------------
# 고정 문구 정의
# -----------------------------
comfort_messages = [
    "힘드셨겠어요, 이런 정보가 도움이 되길 바랍니다 💛",
    "조금이나마 위로가 되었으면 해요 💛",
    "혼자가 아니에요, 언제든 물어봐 주세요 💛"
]
sensitive_triggers = ['죽고 싶', '자살', '극단', '끝내고 싶']
fallback_message = "아직 준비 중인 내용이에요. 궁금한 게 있으면 언제든 물어봐 주세요 💛"

# -----------------------------
# 데이터 파일 로드
# -----------------------------
with open(mental_disorders_path, 'r', encoding='utf-8') as f:
    mental_disorders = json.load(f)
with open(medications_path, 'r', encoding='utf-8') as f:
    medications = json.load(f)
with open(synonyms_path, 'r', encoding='utf-8') as f:
    synonyms = json.load(f)

# -----------------------------
# 약물 동의어 맵 생성
# -----------------------------
def build_medication_synonym_map(medications):
    med_synonyms = {}
    for med in medications:
        med_field = med.get('약물 (일반명/상품명)', '')
        match = re.match(r'(.+?)\s*\((.+?)\)', med_field)
        if match:
            names = [match.group(1).strip(), match.group(2).strip()]
        else:
            names = [med_field.strip()]
        med_synonyms[names[0]] = names
    return med_synonyms

med_synonym_map = build_medication_synonym_map(medications)
for key, values in med_synonym_map.items():
    synonyms[key] = list(set(synonyms.get(key, []) + values))

# -----------------------------
# 오타 교정
# -----------------------------
def correct_input(user_input, known_words):
    words = user_input.split()
    corrected = []
    for word in words:
        matches = difflib.get_close_matches(word, known_words, n=1, cutoff=0.8)
        corrected.append(matches[0] if matches else word)
    return ' '.join(corrected)

# -----------------------------
# 동의어 매칭
# -----------------------------
def match_synonym(user_input, target_key):
    return any(re.search(rf'\b{re.escape(word)}\b', user_input) for word in synonyms.get(target_key, [target_key]))

# -----------------------------
# 챗봇 메인 함수
# -----------------------------
def chatbot(user_input):
    try:
        known_words = list(synonyms.keys()) + [m.get('약물 (일반명/상품명)', '') for m in medications]
        user_input_corrected = correct_input(user_input, known_words)
        user_input_clean = re.sub(r'[^\w\s]', '', user_input_corrected.strip())
        logging.info(f"사용자 입력: {user_input_clean}")

        # 민감 질문 처리
        if any(trigger in user_input_clean for trigger in sensitive_triggers):
            return "⚠️ 위험한 생각이 드신다면 가까운 사람이나 전문가에게 꼭 도움을 요청하세요. 💛 당신은 혼자가 아닙니다."

        # 부작용 질문
        if '부작용' in user_input_clean:
            for med in medications:
                med_field = med.get('약물 (일반명/상품명)', '')
                med_names = med_synonym_map.get(med_field.split()[0], [med_field])
                if any(med_name in user_input_clean for med_name in med_names):
                    return f"💊 {med_field} 부작용:\n- {med.get('부작용 상세', '부작용 정보 없음')}\n{random.choice(comfort_messages)}"
            return f"앗, 해당 약물의 부작용 정보를 아직 찾지 못했어요.\n💊 예: 플루옥세틴, 에스시탈로프람, 자낙스 등 이름으로 질문해 주세요.\n{random.choice(comfort_messages)}"

        # 약물 질문
        med_keywords = ['약', '약물', '치료', '복용', '추천', '먹는 약']
        if any(kw in user_input_clean for kw in med_keywords):
            for disorder in mental_disorders:
                disorder_name = disorder.get('질병', '')
                if disorder_name == '성격장애':
                    continue
                if match_synonym(user_input_clean, disorder_name):
                    treatments = disorder.get('치료제', '').strip() or '정보 없음'
                    symptoms = disorder.get('증상', '').strip() or '정보 없음'
                    return f"💊 {disorder_name}\n- 약물: {treatments}\n📌 특징 증상: {symptoms}\n{random.choice(comfort_messages)}"
            return fallback_message

        # 증상 질문
        for disorder in mental_disorders:
            disorder_name = disorder.get('질병', '')
            if disorder_name == '성격장애':
                continue
            if match_synonym(user_input_clean, disorder_name):
                symptoms = disorder.get('증상', '').strip()
                definition = disorder.get('정의', '').strip()
                return f"🔍 {disorder_name}\n📌 정의: {definition}\n📌 증상: {symptoms}\n{random.choice(comfort_messages)}"

        # fallback
        return fallback_message

    except Exception as e:
        logging.error(f"오류 발생: {str(e)}")
        return "잠시 문제가 생겼어요. 조금 후 다시 시도해 주세요 💛"


In [160]:
test_questions = [
    "플루옥세틴 부작용 뭐야?",
    "에스시탈로프람 먹으면 무슨 부작용 있어?",
    "자낙스 부작용 알려줘",
    "공황장애 약물 뭐 있어?",
    "불안할 때 먹는 약 추천해줘",
    "우울증에 쓰는 약 알려줘",
    "불면증에 좋은 약은 뭐야?",
    "ADHD 약물 추천해줄 수 있어?",
    "알츠하이머 약물 치료법 있어?",
    "양극성장애 약물 뭐가 있어?",
    "불안장애 증상 알려줘",
    "우울증 증상 뭐야?",
    "공황장애 증상 궁금해",
    "경계성 성격장애 특징은?",
    "자꾸 폭식하게 돼. 왜 그래?",
    "최근에 잠이 잘 안 와",
    "기분이 너무 오락가락해",
    "집중이 잘 안 돼",
    "요즘 무기력해",
    "나 요즘 예민해졌어",
    "요즘 너무 불안해",
    "너무 우울해",
    "최근에 너무 충동적이야",
    "나는 정신과 가야 할까?",
    "나 요즘 너무 예민해서 힘들어",
    "아무것도 하기 싫어",
    "요즘 사람 만나는 게 무서워",
    "감정 조절이 안 돼",
    "아무 생각이 없고 멍해",
    "죽고 싶다는 생각이 자꾸 들어"
]

for q in test_questions:
    print(f"Q: {q}")
    print(f"A: {chatbot(q)}\n")


Q: 플루옥세틴 부작용 뭐야?
A: 💊 플루옥세틴 (프로작) 부작용:
- 불안, 불면, 메스꺼움, 두통, 성기능 장애, 식욕 저하, 입 마름, 발한, 설사, 떨림
힘드셨겠어요, 이런 정보가 도움이 되길 바랍니다 💛

Q: 에스시탈로프람 먹으면 무슨 부작용 있어?
A: 💊 에스시탈로프람 (렉사프로) 부작용:
- 메스꺼움, 불안, 어지럼증, 졸음, 불면, 성기능 장애, 입 마름, 발한, 두통, 설사
조금이나마 위로가 되었으면 해요 💛

Q: 자낙스 부작용 알려줘
A: 앗, 해당 약물의 부작용 정보를 아직 찾지 못했어요.
💊 예: 플루옥세틴, 에스시탈로프람, 자낙스 등 이름으로 질문해 주세요.
힘드셨겠어요, 이런 정보가 도움이 되길 바랍니다 💛

Q: 공황장애 약물 뭐 있어?
A: 💊 공황장애
- 약물: SSRI(설트랄린, 파록세틴), 벤조디아제핀(알프라졸람)
📌 특징 증상: 갑작스러운 극심한 공포, 심계항진, 발한, 떨림, 숨가쁨, 흉통, 메스꺼움, 현기증, 비현실감, 죽음에 대한 공포, 발작 후 극도의 피로감
혼자가 아니에요, 언제든 물어봐 주세요 💛

Q: 불안할 때 먹는 약 추천해줘
A: 아직 준비 중인 내용이에요. 궁금한 게 있으면 언제든 물어봐 주세요 💛

Q: 우울증에 쓰는 약 알려줘
A: 💊 우울증
- 약물: SSRI, SNRI, 미르타자핀, 부프로피온
📌 특징 증상: 지속적 우울감, 흥미·즐거움 상실, 체중·식욕 변화, 수면 문제, 피로감, 무가치감, 집중력 저하, 자살 생각
힘드셨겠어요, 이런 정보가 도움이 되길 바랍니다 💛

Q: 불면증에 좋은 약은 뭐야?
A: 아직 준비 중인 내용이에요. 궁금한 게 있으면 언제든 물어봐 주세요 💛

Q: ADHD 약물 추천해줄 수 있어?
A: 아직 준비 중인 내용이에요. 궁금한 게 있으면 언제든 물어봐 주세요 💛

Q: 알츠하이머 약물 치료법 있어?
A: 아직 준비 중인 내용이에요. 궁금한 게 있으면 언제든 물어봐 주세요 💛

Q: 양극성장애 약물 뭐가 있어?
A: 아직 준비 중인 내용이에요. 궁금한 게 있으면